# Imports

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2
import matplotlib 
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0) 
    

In [ ]:
# LIDAR_PACKAGE_PATH = 'mocap_funct.py'
# import sys
# if LIDAR_PACKAGE_PATH not in sys.path: sys.path.append(LIDAR_PACKAGE_PATH)
# import os 
# from LIDAR.nnet import generateTrainingSet, egoh5
# from scripts.utils import print_d
# from kinematics_utils import *

%matplotlib notebook
from mocap_func import *
import h5py 
import matplotlib.pyplot as plt
import numpy as np
import glob
from scipy.ndimage import median_filter
import time

# Reading Mocap Files

In [ ]:
import time

raw_dir ='/Users/alexle/Desktop/Harvard/VAE_mat_files'

matnames = []
errors = []

matnames = glob.glob('temporal_data.mat')
print(matnames)
matfile = 'VAE_mat_files/temporal_data.mat'
# matfile = h5py.File('VAE_mat_files/temporal_data.mat')


In [ ]:
# for matfile in matnames:
#     t1 = time.time()
#     print('Loading '+ matfile.split('/')[-1])
with h5py.File(matfile, 'r') as h5file:
    open_file = h5file['aligned_mean_position']
    keytofind = 'aligned_mean_position'
    temp = list(h5file.keys()) 
    print(temp[0])
    print(list(h5file.keys())[0])
    print(np.array(h5file[keytofind]).shape)
    print(np.array(h5file[keytofind]))



    # tslen = h5file[keytofind][list(h5file[keytofind].keys())[0]].shape[1]            
#     print('\t Time series length : %i. Reading time series...\t%0.02f'%(tslen, time.time()-t1))
#     ts = np.zeros((tslen, 60))
#     mat_bad_bool = np.zeros((tslen, 20), dtype=bool) 
#     for bind, bpart in enumerate(h5file[keytofind]):              
#         print('\t\t%i, %s\t%0.02f'%(bind, bpart, time.time()-t1), end='\r')
#         ts[:,bind*3:(bind+1)*3] = h5file[keytofind][bpart][:,:tslen].T
#         mat_bad_bool[np.where(~np.any(np.abs(ts[:,bind*3:(bind+1)*3])>1e-2, axis=1))[0], bind] = True
# errors.append(np.any(mat_bad_bool, axis=1).sum()*100/tslen)
# print('%s : %0.02f%% error.'%(matfile.split('/')[-1][:-4], errors[-1]))
    # np.save('mocap_ephys_positions/%s.npy'%(matfile.split('/')[-1][:-4]), ts)

# errors = np.array(errors)

# Reading Saved Raw Data (reading at 60FPS), processing and saving.

In [ ]:
for i,mname in enumerate(matnames):
    print('%i\t%s'%(i,mname.split('/')[-1][:-4]))
    t1 = time.time()
    h5 = np.load('mocap_ephys_positions/%s.npy'%(mname.split('/')[-1][:-4]))[::5]
    print('\t Raw file loaded in %i seconds. Fixing bad indices now.'%(time.time()-t1))

    t = time.time()
    h5[np.isnan(h5)] = 0.0
    goodinds = np.any(np.abs(h5).reshape((-1,20,3))>1e-2, axis=2)
    goodbefore = np.all(goodinds, axis=1).sum()*100/goodinds.shape[0]
    for bpart in range(goodinds.shape[1]):
        badinds = np.where(~goodinds[:,bpart])[0]
        if badinds.shape[0]==0:
            continue
        badinds = np.split(badinds, np.where(np.ediff1d(badinds)!=1)[0]+1)
        fixed = 0
        total = 0

        for inds in badinds:
            total += len(inds)
            if len(inds) < 120:
                if inds[-1]+1 == h5.shape[0]:
                    continue
                fixed += len(inds)
                difftofill = (h5[inds[-1]+1, bpart*3:(bpart+1)*3] - h5[inds[0]-1, bpart*3:(bpart+1)*3])/(len(inds)+1)
                h5[inds, bpart*3:(bpart+1)*3] = h5[inds[0]-1, bpart*3:(bpart+1)*3] + np.multiply(difftofill.reshape((3,1)), np.arange(1,len(inds)+1)).T
#         print('Video %i, bpart %i, fixed %i of %i.'%(i, bpart, fixed, total))
    print('\t Fixed bad indices in %i seconds. Saving processed h5 now.'%(time.time()-t))
    t = time.time()
    np.save('mocap_ephys_positions/%s_processed.npy'%(mname.split('/')[-1][:-4]), h5)
    print('\t Saved in %i seconds.'%(time.time()-t))
    
    goodinds = np.any(np.abs(h5).reshape((-1,20,3))>1e-2, axis=2)
    goodafter = np.all(goodinds, axis=1).sum()*100/goodinds.shape[0]

    m, s = divmod(h5.shape[0]/60, 60)
    h, m = divmod(m, 60)
    print('Good before, after : %0.01f%%, %0.01f%%  \t length : %i:%i:%0.01f. Time taken to process : %0.01f seconds.'%(goodbefore, goodafter, h, m, s, (time.time()-t1)))
    print('\n')



# Reading Processed h5s

In [ ]:
h5s = []
goodframe_idxs = []
goods = []
for i,mname in enumerate(matnames):
    print('%i\t%s'%(i,mname.split('/')[-1][:-4]))
    t1 = time.time()

    h5 = np.load('mocap_ephys_positions/%s_processed.npy'%(mname.split('/')[-1][:-4]))
    h5s.append(h5)
    ginds = np.any(np.abs(h5).reshape((-1,20,3))>1e-2, axis=2)
    goodframe_idxs.append(ginds)
    goods.append(np.all(ginds, axis=1).sum()*100/ginds.shape[0])
    
    m, s = divmod(h5.shape[0]/60, 60)
    h, m = divmod(m, 60)
    print('%s \t %0.01f%% good \t length : %i:%i:%0.01f '%(mname.split('/')[-1], goods[-1], h, m, s))
    print('\t Done : %0.1f seconds.'%(time.time()-t1))
h5s = np.array(h5s)
print(h5s)

# Egocenter h5s

## Save Ego h5s

In [ ]:
egoh5s = []
for i in range(len(h5s)):
    print(i, end='\r')
    mname = matnames[i]
    h5 = h5s[i]
    try:
        tqdm._instances.clear()
    except:
        pass
    h5 = h5.reshape((-1,20,3))
    h5 = np.append(h5, h5[:,[4,6],:].mean(axis=1).reshape((-1,1,3)), axis=1)
    h5 = h5.reshape((-1,h5.shape[1]*h5.shape[2]))
    
    eh5 = egoh5(h5, ndims=3, bindcenter=19, b1=20, b2=18, silent=True)[:,:57]
    np.save('mocap_ephys_positions/%s_processed_eh5.npy'%(mname.split('/')[-1][:-4]), eh5)
    egoh5s.append(eh5)

# egoh5s = np.array(egoh5s)
# print(egoh5s.shape)


## Read Ego h5s

In [ ]:
# egoh5s = []
for i,mname in enumerate(matnames):
    print('%i\t%s'%(i,mname.split('/')[-1][:-4]))
    t1 = time.time()

    eh5 = np.load('mocap_ephys_positions/%s_processed_eh5.npy'%(mname.split('/')[-1][:-4]))
    if np.any(np.isnan(eh5)):
        print('\t has nans.')
    egoh5s.append(eh5)
    print('\t Done : %0.1f seconds.'%(time.time()-t1))
    
egoh5s = np.array(egoh5s)
print(egoh5s.shape)

## Check Ego h5s for nans

In [ ]:
for i,mname in enumerate(matnames):
    print('%i\t%s'%(i,mname.split('/')[-1][:-4]))
    t1 = time.time()

    eh5 = np.load('mocap_ephys_positions/%s_processed_eh5.npy'%(mname.split('/')[-1][:-4]))
    eh5[np.isnan(eh5)] = 0.0
    np.save('mocap_ephys_positions/%s_processed_eh5.npy'%(mname.split('/')[-1][:-4]), eh5)
    
    badinds = np.isnan(eh5)
    if np.any(badinds):
        print('\t\t has nans. %i/%i'%(np.sum(np.any(badinds, axis=1)), eh5.shape[0]))
        h5 = np.load('mocap_ephys_positions/%s_processed.npy'%(mname.split('/')[-1][:-4]))
        badindsh5 = np.isnan(h5)
        print('\t\t Processed has nans. %i/%i'%(np.sum(np.any(badindsh5, axis=1)), h5.shape[0]))
    print('\t Done : %0.1f seconds.'%(time.time()-t1))



In [ ]:
bpartnames = ['ArmL', 'ArmR', 'ElbowL', 'ElbowR', 'HeadB', 'HeadF', 'HeadL', 'HipL', 
              'HipR', 'KneeL', 'KneeR', 'Offset1', 'Offset2', 'ShinL', 'ShinR', 'ShoulderL', 
              'ShoulderR', 'SpineF', 'SpineL', 'SpineM']

connections = [[5, 4, 6], [4,17,19,18], [11,12], [0,2,15,17,16,3,1], [13,9,7,18,8,10,14]]
connections2 = [[5, 4, 6], [4,17,18], [11,12], [0,2,15,17,16,3,1], [13,9,7,18,8,10,14]]


# Check poses

In [ ]:
%matplotlib inline
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')

# h5s = np.array(h5s)
# x = np.array(h5s[0][1][0::3])
# y = np.array(h5s[0][1][1::3])
# z = np.array(h5s[0][1][2::3])
# ax.scatter(x,y,z, color='#ff5e5e', s =10, marker='x')
# ax.plot()

%matplotlib inline
fig, ax = plt.subplots(figsize=(4,4))
vind = 0
t = 100000
temp = np.array(h5s)
print(temp.shape)
for i in range(20):
    ax.scatter(h5s[vind][t,3*i],h5s[vind][t,3*i+1],marker='$'+str(i)+'$',s=200)
for conn in connections:
    ax.plot(h5s[vind][t].reshape((20,3))[conn,0], h5s[vind][t].reshape((20,3))[conn,1])
    
    

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(4,4))
vind = 0
t = 100000

for i in range(19):
    ax.scatter(egoh5s[vind][t,3*i], egoh5s[vind][t,3*i+1],marker='$'+str(i)+'$',s=200)
for conn in connections2:
    ax.plot(egoh5s[vind][t].reshape((19,3))[conn,0], egoh5s[vind][t].reshape((19,3))[conn,1])
    

In [ ]:
vind = 0
rawh5 = h5s[0].reshape((-1,20,3))
badinds = np.all(goodframe_idxs[vind], axis=1)

%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(4,4))
ax1 = fig.add_subplot(111, projection='3d')
# fig.subplots_adjust(0,0,1,1,0,0)
plt.ion()
fig.show()
fig.canvas.draw()
count = 0
for t in np.setdiff1d(np.arange(rawh5.shape[0]), np.where(np.all(goodframe_idxs[vind], axis=1))[0]):
    ax1.clear()
    for conn in connections:
        ax1.plot(rawh5[t,conn,0],rawh5[t,conn,1], rawh5[t,conn,2], 'o-', alpha=0.7)
    ax1.set_xlim([-200, 200])
    ax1.set_ylim([-200, 200])
    ax1.set_zlim([-50, 150])
    ax1.view_init(10, count)
    count += 1
#     ax1.grid('off')
#     ax1.axis('off')
    if np.all(goodframe_idxs[vind][t]):
        ax.set_title('Good %i'%t, color='royalblue')
    else:
        ax.set_title('Good %i'%t, color='firebrick')

    fig.canvas.draw()        

In [ ]:
rawh5 = egoh5s[vind].reshape((-1,19,3))

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(5,5))
ax1 = fig.add_subplot(111, projection='3d')
# fig.subplots_adjust(0,0,1,1,0,0)
plt.ion()
fig.show()
fig.canvas.draw()

for t in np.arange(0,rawh5.shape[0],50):#range(0,10000,200)        
    ax1.clear()
    for conn in connections2:
        ax1.plot(rawh5[t,conn,0],rawh5[t,conn,1], rawh5[t,conn,2], 'o-', alpha=0.7)
    ax1.set_xlim([-200, 200])
    ax1.set_ylim([-200, 200])
    ax1.set_zlim([-50, 150])
    ax1.view_init(75, 90)
#     ax1.grid('off')
#     ax1.axis('off')
    if t in inds:
        ax.set_title('Fixed %i'%t)
    else:
        ax.set_title('Original %i'%t)

    fig.canvas.draw()        

# Data Summary

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12,5))


barWidth = 0.04
colors = matplotlib.cm.hsv(np.linspace(0,1,len(h5s)))

for j,goodinds in enumerate(goodframe_idxs):
    counts = np.histogram(np.sum(~goodinds, axis=1), bins = np.arange(21), density=False)
    ax.bar(np.arange(1,21)-0.5+j*barWidth, counts[0], color=colors[j], width=barWidth, alpha=0.8)
ax.set_xlabel('#Number of simultaneous errors')
ax.set_ylabel('# of Frames (at 60 FPS)')
ax.set_title('Colors represent different datasets')
# ax.set_yscale('log')
_ = ax.set_xticks(np.arange(1, 21, 1.0).astype('int'))


In [ ]:
colors = matplotlib.cm.gist_ncar(np.linspace(0,0.9,len(h5s)))
%matplotlib inline

fig, (ax, ax2) = plt.subplots(2,1,figsize=(12.0,10.0), num=2)
fig.subplots_adjust(0.05,0.05,0.98,0.95,0,0.2)
barWidth = 0.04

fracgood = []
for j,h5 in enumerate(h5s):
    goodinds = goodframe_idxs[j]
    h5 = h5
    cnot1 = (np.sum(~goodinds, axis=0)/h5.shape[0])
    ax.bar(np.arange(20)+j*barWidth, cnot1, color=colors[j], width=barWidth, alpha=0.8)
    fracgood.append(np.all(goodinds, axis=1).sum()/h5.shape[0])

ax.set_xticks(np.arange(20)+barWidth*len(h5s)/2.0)
ax.set_xticklabels(bpartnames, rotation=45)
ax.set_title("Fraction of frames bad for each bodypart.")
# ax.set_ylim([0.7,1.02])
ax.grid()
ax2.bar(np.arange(len(h5s)), fracgood, color=colors, width=0.4, linewidth=4.0, edgecolor='k', alpha=0.8)
ax2.set_xlabel('Video #')
ax2.set_ylabel('Fraction of good frames in each video.')
ax2.set_ylim([0.,1])
ax2.grid()



In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12.0,4.0))
goodsplits = []
for j,h5 in enumerate(h5s):
    goodinds = goodframe_idxs[j]
    goodinds = np.where(np.all(goodinds, axis=1))[0]
    splits = np.array(np.split(goodinds, np.where(np.ediff1d(goodinds)!=1)[0]+1))
    split_lens = [len(split) for split in splits]
    goodsplits.append([split for split in splits if len(split)>30000 ])
    ax.hist(split_lens, bins=np.arange(0,np.max(split_lens)+500, step=500), alpha=0.8, label=str(j))
_ = ax.set_yscale('log')
# ax.legend()

ax.grid()

In [ ]:
%matplotlib inline
import matplotlib
min_length = 100000
goodinds_thres = []

colorslist = matplotlib.cm.gist_ncar(np.linspace(0,0.9,len(h5s)))
# np.random.shuffle(colors)
fig, ax = plt.subplots(figsize=(12.0, 15.0))
fig.subplots_adjust(0.05,0.05,0.95,0.95,0,0)
for j,h5 in enumerate(h5s):
    print('Loading....%0.2f%%'%(j*100/len(h5s)), end='\r')
    h5 = h5
    goodinds = goodframe_idxs[j]
    goodinds = np.where(np.all(goodinds, axis=1))[0]
    goodinds = np.array(np.split(goodinds, np.where(np.ediff1d(goodinds)!=1)[0]+1))
    vel = np.linalg.norm(np.diff(np.mean(h5.reshape((h5.shape[0], int(h5.shape[1]/3), 3)), axis=1), axis=0), axis=1)
    vel = np.concatenate(([vel[0]], vel), axis=0)
    # vel = np.clip(vel, 0, 1)/1.0
    for inds in goodinds:   
        ax.scatter(inds, np.ones_like(inds)*j, color=colorslist[j], marker='|', s=10+vel[inds]*5000/vel.max())
    ax.text(h5.shape[0]+2000, j-0.02, '%0.1f%% of %0.1f seconds'%(fracgood[j]*100, h5.shape[0]/60.0), fontweight='bold', color='k')
    for g in goodinds:
        if g.shape[0]>min_length:
            rect = matplotlib.patches.Rectangle((g[0], j-0.4),g[-1]-g[0],0.8,linewidth=1,edgecolor='k',facecolor='none')
            ax.add_patch(rect)
    goodinds_thres.append([g for g in goodinds if g.shape[0]>min_length])
    print('%i total frames are %i'%(j, np.sum([len(g) for g in goodinds if g.shape[0]>min_length])), end='\r')
ax.set_yticks(np.arange(len(h5s)))
ax.set_title("Box lengths > %0.1f seconds"%(min_length/60.0))
print('Total frames in the box : %i'% np.sum([g.shape[0] for ginds in goodinds_thres for g in ginds]))

In [ ]:
lengthlist = [[int(len(g)/3600) for g in ginds] for ginds in goodinds_thres]
print('Length of good segments in mins :')
lengthlist

# Save these segments for VAE

In [ ]:
outpath = '/mnt/HFSP_Data/scripts/Marshall_Data/VAE_RNN/mocap_clean_eh5/%s'
for mname, eh5, ginds in zip(matnames, egoh5s, goodinds_thres):
    if not ginds:
        continue
    glens = [len(g)/3600 for g in ginds]
    g = ginds[np.argmax(glens)]

    mname = mname.split('/')[-1][22:-4]+'_%i_%i.npy'%(g[0], g[-1])
    print(mname)
    
    np.save(outpath%mname, eh5[g[0]:g[-1]])
    
